# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [2]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_coding_vs_intergenomic_seqs 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

/Users/katarina/Documents/repositories/GitHub/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2022-06-02 23:22:16.313188: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

/Users/katarina/Documents/repositories/GitHub/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_coding_vs_intergenomic_seqs` has 2 classes: coding_seqs, intergenomic_seqs.

All lengths of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
coding_seqs,37500,12500
intergenomic_seqs,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


In [7]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

In [8]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
vectorize_layer.get_vocabulary()

['', '[UNK]', 'a', 't', 'g', 'c']

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [10]:
model = get_model(NUM_CLASSES)

In [11]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
1172/1172 [==============================] - 55s 44ms/step - loss: 0.3490 - binary_accuracy: 0.8409 - f1_score: 0.8301
Epoch 2/10
1172/1172 [==============================] - 61s 52ms/step - loss: 0.2907 - binary_accuracy: 0.8744 - f1_score: 0.8683
Epoch 3/10
1172/1172 [==============================] - 59s 50ms/step - loss: 0.2757 - binary_accuracy: 0.8816 - f1_score: 0.8765
Epoch 4/10
1172/1172 [==============================] - 57s 48ms/step - loss: 0.2652 - binary_accuracy: 0.8871 - f1_score: 0.8824
Epoch 5/10
1172/1172 [==============================] - 60s 51ms/step - loss: 0.2587 - binary_accuracy: 0.8907 - f1_score: 0.8864
Epoch 6/10
1172/1172 [==============================] - 49s 42ms/step - loss: 0.2534 - binary_accuracy: 0.8931 - f1_score: 0.8891
Epoch 7/10
1172/1172 [==============================] - 44s 37ms/step - loss: 0.2505 - binary_accuracy: 0.8945 - f1_score: 0.8905
Epoch 8/10
1172/1172 [==============================] - 55s 47ms/step - loss: 0.2463 - bin

## Evaluation on the test set

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

391/391 [==============================] - 4s 10ms/step - loss: 0.3011 - binary_accuracy: 0.8552 - f1_score: 0.8368


[0.3011115491390228, 0.8551599979400635, 0.8367963433265686]